# Einstieg

Die folgende Datei wurde über jQAssistant

<pre>
jqassistant scan -f xml:document::Cloner_newInstance.xml
</pre>

in Neo4J eingelesen.

In [7]:
with open (r'C:\Development\repos\jqassistant_jprofiler_xml\cpu_hot_spots_newInstance\Cloner_newInstance.xml') as log:
    [print(line[:120] + "...") for line in log.readlines()[:10]]

<?xml version="1.0" encoding="UTF-8"?>
...
<tree type="Hot Spots" viewFilters="com.rits" threadSelection="All thread groups" threadStatus="Runnable" aggregationLev...
  <hotspot leaf="false" class="com.rits.cloning.Cloner" methodName="newInstance" methodSignature="(Ljava/lang/Class;)Lja...
    <node leaf="false" class="com.rits.cloning.Cloner" methodName="cloneObject" methodSignature="(Ljava/lang/Object;Ljav...
      <node leaf="false" class="com.rits.cloning.Cloner" methodName="cloneInternal" methodSignature="(Ljava/lang/Object;...
        <node leaf="false" class="com.rits.cloning.Cloner" methodName="cloneObject" methodSignature="(Ljava/lang/Object;...
          <node leaf="false" class="com.rits.cloning.Cloner" methodName="cloneInternal" methodSignature="(Ljava/lang/Obj...
            <node leaf="false" class="com.rits.cloning.Cloner" methodName="cloneObject" methodSignature="(Ljava/lang/Obj...
              <node leaf="false" class="com.rits.cloning.Cloner" methodName="cloneInterna

 Der Neo4J-Server wurde mit 

<pre>
jqassistant server
</pre>

mit den Standardeinstellungen gestartet.

Wichtig ist, dass der Server unter der Oracle JVM läuft, was z. B. mit
<pre>
set PATH=C:\Development\apps\Java\jdk1.8.0_25\bin;%PATH%
</pre>
gesetzt werden kann.

# Erstelle Konten für eine bessere Analyse

In [8]:
from py2neo import Graph
graph = Graph()
from py2neo.packages.httpstream import http
http.socket_timeout = 240

## 0. Vorbereitung

### Lösche alle evtl. vorhandenen Konten und Beziehungen

In [9]:
query = """
MATCH (c:Call)-[r:CALLED]->()
DELETE r
RETURN COUNT(r)
"""
graph.run(query).data()

[{'COUNT(r)': 0}]

In [10]:
query = """
MATCH (n:Call)-[r:CREATED_FROM]->(n:Element)
DELETE r, n
RETURN COUNT(r), COUNT(n)
"""
graph.run(query).data()

[{'COUNT(n)': 0, 'COUNT(r)': 0}]

### Setze Index

In [11]:
query = """
CREATE INDEX ON :Element(name)
"""
#graph.run(query).data()

## 1. Call-Nodes aus XML-Elements erzeugen

In [12]:
query = """
MATCH (n:Element),
(n)-[:HAS_ATTRIBUTE]->(classAttribut:Attribute {name : "class"}),
(n)-[:HAS_ATTRIBUTE]->(methodAttribut:Attribute {name : "methodName"}),
(n)-[:HAS_ATTRIBUTE]->(countAttribut:Attribute {name : "count"}),
(n)-[:HAS_ATTRIBUTE]->(timeAttribut:Attribute {name : "time"})
WHERE n.name = "hotspot" OR n.name = "node"
CREATE 
   (x:Call {
      class: classAttribut.value, 
      method: methodAttribut.value, 
      count: toFloat(countAttribut.value), 
      time: toFloat(timeAttribut.value)})-[r:CREATED_FROM]->(n)
RETURN COUNT(x), COUNT(r)
"""
graph.run(query).data()

[{'COUNT(r)': 136586, 'COUNT(x)': 136586}]

## 2. Beziehung zu allen Calls untereinander herstellen

In [13]:
query="""
MATCH (outerCall:Call)-[:CREATED_FROM]->(outerNode)-[:HAS_ELEMENT]->(innerNode)<-[:CREATED_FROM]-(innerCall:Call)
CREATE (outerCall)-[r:CALLED]->(innerCall)
RETURN COUNT(r)
"""
graph.run(query).data()

SocketError: timed out

## 3. Hotspots markieren

In [ ]:
query="""
MATCH (x:Call)-[r:CREATED_FROM]->(n:Element { name: "hotspot"})
SET x:HotSpot
REMOVE x:Call
RETURN COUNT(x), COUNT(r)
"""
graph.run(query).data()

# Analysen

In [ ]:
import pandas as pd

## TOP-Auslöser der HotSpots

In [ ]:
query="""

"""
auswertung = pd.DataFrame(graph.run(query).data())
auswertung.head()

In [ ]:
df['bk'] = df['b'].apply(lambda x : x.split("#")[0])
df.head()

In [ ]:
grouped= df.groupby('bk').sum().sort_values(by="Zeitdauer", ascending=False)
grouped.head()

In [ ]:
top = grouped.head(10)
top 

In [ ]:
andere_hotspots = grouped[~grouped['Zeitdauer'].isin(top.index)]
andere_hotspots.head()

In [ ]:
name_andere = "" + str(len(andere_hotspots)) + " weitere Klassen"
andere_hotspots_summiert = pd.DataFrame(andere_hotspots.sum(), columns=[name_andere]).T
andere_hotspots_summiert.head()

In [ ]:
ergebnis_top10_hotspots = top.append(andere_hotspots_summiert)
ergebnis_top10_hotspots['Verweildauer in Minuten'] = top['Zeitdauer'].apply(
    lambda p: '{:.2f}'.format(p/1000/1000/60))
ergebnis_top10_hotspots.index.name="Klassenname"
ergebnis_top10_hotspots[['Verweildauer in Minuten']]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np

def plotteDaten(series, titel):

    plt.style.use('fivethirtyeight')
    plt.figure(facecolor='white')

    ax = series.plot(
        kind='pie', 
        colors=cm.Spectral(np.linspace(0, 1, len(series))), 
        figsize=(10,10), 
        legend=None, 
        autopct='%1.0f%%',
        pctdistance=1.1, 
        labeldistance=1.2,
        fontsize=10,
        title=titel)
    ax.set_ylabel("")
    fig = ax.get_figure()
    plt.savefig("output/" + titel + ".svg", facecolor=fig.get_facecolor(), bbox_inches="tight")

In [ ]:
plotteDaten(top['Zeitdauer'], u"Cloner-Verweildauer")